<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [77]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

import json
import string
import random
import re # Regular Expressions (regex)
import urllib.request

import numpy as np
# Para leer y parsear el texto en HTML de wikipedia
import bs4 as bs

import nltk
# Descargar el diccionario



### Datos
Utilizaremos como dataset conversaciones de la serie los Simpsons.

In [78]:
df=pd.read_csv("/content/simpsons_script_lines.csv",low_memory=False)

In [79]:
df=df.dropna()

### 1 - Preprocesamiento

In [80]:
#Acomodo todas las oraciones en una lista, quitando puntuacion

def remove(text):
  text=str(text).lower()
  return re.sub(r'[^\w\s]', '', text).split()

sentence_tokens=[remove(x) for x in df["spoken_words"].values if len(x)]

In [81]:
# Demos un vistazo
sentence_tokens[995:1000]

[['maggie'],
 ['bart', 'i', 'asked', 'you', 'to', 'watch', 'your', 'sister'],
 ['i', 'tried', 'to', 'stop', 'her', 'but', 'she', 'overpowered', 'me'],
 ['who', 'stole', 'my', 'shirt', 'who', 'stole', 'my', 'shirt'],
 ['i', 'washed', 'it']]

### 2 - Crear los vectores (word2vec)

In [82]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [83]:
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [84]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [85]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 131708


In [86]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 11707


### 3 - Entrenar el modelo generador

In [87]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=100,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 7946465.0
Loss after epoch 1: 6324143.0
Loss after epoch 2: 6145012.0
Loss after epoch 3: 6007852.0
Loss after epoch 4: 5918764.0
Loss after epoch 5: 6240572.0
Loss after epoch 6: 6202912.0
Loss after epoch 7: 6069992.0
Loss after epoch 8: 5959484.0
Loss after epoch 9: 5846244.0
Loss after epoch 10: 4758272.0
Loss after epoch 11: 1374560.0
Loss after epoch 12: 1375144.0
Loss after epoch 13: 1373352.0
Loss after epoch 14: 1366440.0
Loss after epoch 15: 1359920.0
Loss after epoch 16: 1353872.0
Loss after epoch 17: 1342488.0
Loss after epoch 18: 1336304.0
Loss after epoch 19: 1327360.0
Loss after epoch 20: 1316496.0
Loss after epoch 21: 1309592.0
Loss after epoch 22: 1300648.0
Loss after epoch 23: 1295952.0
Loss after epoch 24: 1286576.0
Loss after epoch 25: 1276384.0
Loss after epoch 26: 1264912.0
Loss after epoch 27: 1257752.0
Loss after epoch 28: 1245696.0
Loss after epoch 29: 1239824.0
Loss after epoch 30: 1230728.0
Loss after epoch 31: 1221496.0
Loss after epoch 3

(95009323, 129925700)

In [102]:
#guardar el modelo
w2v_model.save('model-simpsons-nlp.bin')

In [89]:
# Load the model
#import gensim
#w2v_model = gensim.models.Word2Vec.load('model-simpsons-nlp.bin')


### 4 - Ensayar

In [103]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["burns"], topn=5)

[('mr', 0.4935748875141144),
 ('montgomery', 0.3828044831752777),
 ('burnss', 0.37974125146865845),
 ('rafting', 0.373269259929657),
 ('welly', 0.35727304220199585)]

In [104]:
w2v_model.wv.most_similar(positive=["bart"], topn=5)

[('kay', 0.35072818398475647),
 ('milhouse', 0.34625452756881714),
 ('rebuttal', 0.34220921993255615),
 ('hidilly', 0.3407613933086395),
 ('toottoot', 0.33982640504837036)]

In [105]:
w2v_model.wv.most_similar(positive=["homer"], topn=5)

[('simpson', 0.41499626636505127),
 ('tata', 0.3883100748062134),
 ('simp', 0.36873742938041687),
 ('dad', 0.36840495467185974),
 ('slimu', 0.3587924540042877)]

In [106]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["krusty"], topn=5)

[('clown', 0.3718061149120331),
 ('klown', 0.3570955693721771),
 ('maitre', 0.3284519612789154),
 ('kamp', 0.30465832352638245),
 ('whoops', 0.3043697774410248)]

In [107]:
w2v_model.wv.most_similar(positive=["marge"], topn=5)

[('daaad', 0.3731583058834076),
 ('homer', 0.3584832549095154),
 ('sylvia', 0.34055328369140625),
 ('homie', 0.33837249875068665),
 ('hoochie', 0.32837820053100586)]

In [108]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["simpson"], topn=5)

[('apartment', 0.09350478649139404),
 ('smaller', 0.08921615779399872),
 ('sallright', 0.08321058005094528),
 ('cheap', 0.07480017840862274),
 ('broadway', 0.05807023122906685)]

In [100]:
w2v_model.wv.most_similar(negative=["bart"], topn=5)

[('homeless', 0.0853266641497612),
 ('network', 0.06982976943254471),
 ('sallright', 0.054292142391204834),
 ('firing', 0.051582541316747665),
 ('eastern', 0.03598767891526222)]

In [117]:
w2v_model.wv.most_similar(negative=["otto"], topn=5)

[('stable', 0.0814487561583519),
 ('capsule', 0.07583986967802048),
 ('whether', 0.06643354147672653),
 ('acceptance', 0.065835140645504),
 ('worn', 0.06323283165693283)]

In [118]:
w2v_model.wv.most_similar(negative=["apu"], topn=5)

[('disorder', 0.09113717079162598),
 ('sinkhole', 0.08909886330366135),
 ('toward', 0.08502352237701416),
 ('vital', 0.07961184531450272),
 ('drops', 0.07094001024961472)]

In [119]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(negative=["lisa"], topn=5)

[('remains', 0.08451435714960098),
 ('size', 0.07988189905881882),
 ('filter', 0.05832090228796005),
 ('booze', 0.054347410798072815),
 ('valley', 0.048633936792612076)]

In [120]:
w2v_model.wv.most_similar(negative=["ralph"], topn=5)

[('jet', 0.09406711161136627),
 ('cue', 0.09287181496620178),
 ('learning', 0.07445380091667175),
 ('tartar', 0.07420462369918823),
 ('guide', 0.073801189661026)]

### 5 - Visualizar agrupación de vectores

In [97]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model):
    num_dimensions = 2

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [98]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.